# 使用嵌入进行文档搜索 
## 概览
此示例演示了如何使用 Gemini API 创建嵌入，以便执行文档搜索。您将使用 Python 客户端库构建字词嵌入，以便将搜索字符串或问题与文档内容进行比较。
在本教程中，您将使用嵌入对一组文档执行文档搜索，以提出与 Google 汽车相关的问题。

## 设置
首先，下载并安装 Gemini API Python 库。

In [ ]:
!pip install -U -q google.generativeai

In [3]:
import textwrap
import numpy as np
import pandas as pd
import os

import google.generativeai as genai

from dotenv import load_dotenv
load_dotenv()

from IPython.display import Markdown

### 获取 API 密钥

In [4]:
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [5]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


## 嵌入生成
在本部分中，您将了解如何使用 Gemini API 中的嵌入为一段文本生成嵌入。

In [6]:
title = "The next generation of AI for developers and Google Workspace"
sample_text = ("Title: The next generation of AI for developers and Google Workspace"
    "\n"
    "Full article:\n"
    "\n"
    "Gemini API & Google AI Studio: An approachable way to explore and prototype with generative AI applications")

model = 'models/text-embedding-004'
embedding = genai.embed_content(model=model,
                                content=sample_text,
                                task_type="retrieval_document",
                                title=title)

print(embedding)

{'embedding': [-0.0021609084, -0.0031644478, -0.060120754, -0.0071218493, 0.0008775481, 0.040581927, 0.04457149, 0.03552469, -0.04746538, 0.008888608, -0.02795826, 0.011335696, -0.0024438712, 0.0030851837, -0.018796137, -0.055550937, 0.03142646, 0.0006549187, -0.11370059, 0.063708074, -0.021750016, -0.021367034, -0.09982074, -0.008604756, -0.03330058, -0.012815642, 0.07153146, 0.037064772, 0.022970116, 0.043331202, 0.010670608, 0.040685344, 0.036361404, -0.036222056, -0.01779936, -0.014820963, 0.0053205034, -0.017382707, 0.07044941, 0.002021254, -0.018208738, 0.01755808, 0.006493214, 0.12724239, -0.023805201, 0.010057808, -0.0006948924, 0.070856266, -0.05645728, 0.018311134, 0.090462275, 0.021575555, -0.06656089, 0.026865067, -0.0034812444, -0.0011228676, -0.06535635, -0.0018169138, 0.08672994, 0.028747609, -0.024817271, 0.004653876, -0.058998518, 0.032061696, -0.022604035, -0.015454275, -0.013758674, 0.021129983, -0.047893394, 0.02573244, 0.01302824, -0.018002003, -0.039879415, 0.0234

## 构建嵌入数据库
以下是用于构建嵌入数据库的三个示例文本。您将使用 Gemini API 为每个文档创建嵌入。将它们转换为 DataFrame，以获得更好的可视化效果。

In [7]:
DOCUMENT1 = {
    "title": "Operating the Climate Control System",
    "content": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}
DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}
DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [8]:
## 将字典的内容整理到 DataFrame 中，以获得更好的可视化效果。
df = pd.DataFrame(documents)
df.columns = ['Title', 'Text']
df

,Title,Text
0,Operating the Climate Control System,Your Googlecar has a climate control system th...
1,Touchscreen,Your Googlecar has a large touchscreen display...
2,Shifting Gears,Your Googlecar has an automatic transmission. ...


In [9]:
## 获取每个文本正文的嵌入。将此信息添加到 DataFrame 中。
# Get the embeddings of each text and add to an embeddings column in the dataframe
def embed_fn(title, text):
  return genai.embed_content(model=model,
                             content=text,
                             task_type="retrieval_document",
                             title=title)["embedding"]

df['Embeddings'] = df.apply(lambda row: embed_fn(row['Title'], row['Text']), axis=1)
df

,Title,Text,Embeddings
0,Operating the Climate Control System,Your Googlecar has a climate control system th...,"[0.01696385, 0.0074500814, -0.03125627, -0.014..."
1,Touchscreen,Your Googlecar has a large touchscreen display...,"[0.008353345, 0.029304748, -0.04979913, -0.038..."
2,Shifting Gears,Your Googlecar has an automatic transmission. ...,"[0.026820986, 0.012146816, -0.021441666, 0.013..."


## 包含问答功能的文档搜索
现在，嵌入已生成，接下来让我们创建一个问答系统来搜索这些文档。您将提出一个有关超参数调节的问题，创建问题的嵌入，并将其与 DataFrame 中的嵌入集合进行比较。
问题的嵌入将是一个矢量（浮点值列表），将与使用点积的文档矢量进行比较。从 API 返回的这个矢量已经标准化。点积表示两个向量在方向上的相似度。
点积的值可介于 -1 和 1 之间（含 -1 和 1）。如果两个向量之间的点积为 1，那么这两个向量的方向相同。如果点积值为 0，则这些向量彼此正交或无关。最后，如果点积为 -1，则向量指向相反方向且彼此不相似。
请注意，使用新嵌入模型 (embedding-001) 时，对于用户查询，将任务类型指定为 QUERY；对于嵌入文档文本，请将任务类型指定为 DOCUMENT。

In [10]:
query = "How do you shift gears in the Google car?"
model = 'models/text-embedding-004'

request = genai.embed_content(model=model,
                              content=query,
                              task_type="retrieval_query")

In [11]:
## 使用 find_best_passage 函数计算点积，然后按照从大到小的点积值对 DataFrame 进行排序，以便从数据库中检索相关段落。
def find_best_passage(query, dataframe):
  """
  Compute the distances between the query and each document in the dataframe
  using the dot product.
  """
  query_embedding = genai.embed_content(model=model,
                                        content=query,
                                        task_type="retrieval_query")
  dot_products = np.dot(np.stack(dataframe['Embeddings']), query_embedding["embedding"])
  idx = np.argmax(dot_products)
  return dataframe.iloc[idx]['Text'] # Return text from index with max value

In [12]:
## 查看数据库中最相关的文档：
passage = find_best_passage(query, df)
passage

'Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions.'

## 问答应用
我们来尝试使用文本生成 API 来创建问与系统。在下面输入您自己的自定义数据，以创建简单的问答示例。您仍然会使用点积作为相似度指标。

In [13]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [14]:
prompt = make_prompt(query, passage)
print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below.   Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.   However, you are talking to a non-technical audience, so be sure to break down complicated concepts and   strike a friendly and converstional tone.   If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: 'How do you shift gears in the Google car?'
  PASSAGE: 'Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for drivi

In [15]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-exp-1114


In [16]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')
answer = model.generate_content(prompt)

ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).